# [5주차] 기본 과제

외부 블로그의 정보와 함께 챗봇 구현 하기

## 목표
LangChain으로 개발한 RAG를 다음 블로그의 정보와 연동.

요구사항
- RAG internet source https://spartacodingclub.kr/blog/all-in-challenge_winner 로 설정하고 적절하게 대답 할 수 있도록 만들기

In [1]:
!pip install -qU langchain langchain-community langchain-chroma langchain-openai bs4 python-dotenv

In [3]:
import os
import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from dotenv import load_dotenv

load_dotenv()

print(os.getenv("ENVTEST"))

test


실습 코드는 웹 페이지에서 css class post-header (글의 제목, 정보), post-title (제목), post-content (글 내용) 이렇게 파싱해서 WebBaseLoader를 사용했다.

과제에서도 비슷하게 
- 제목 파트 (제목과 글의 정보들)
- 본문 파트 (글 내용)

이렇게 하면 될거 같아서 class 를 직접 파헤쳐 보았다.

카테고리: css-51sl9e
제목: css-j3idia
정보: css-140vytu

본문: editedContent

In [10]:
loader = WebBaseLoader(
    web_paths=("https://spartacodingclub.kr/blog/all-in-challenge_winner",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("css-51sl9e", "css-j3idia", "css-140vytu", "editedContent")
        )
    ),
)
docs = loader.load()

In [11]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(
    documents=splits,
    embedding=OpenAIEmbeddings()
)

In [12]:
retriever = vectorstore.as_retriever()

user_msg = "ALL-in 코딩 공모전 수상작들을 요약해줘"
retrieved_docs = retriever.invoke(user_msg)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)
prompt = hub.pull("rlm/rag-prompt")

user_prompt = prompt.invoke({"context": format_docs(retrieved_docs), "question": user_msg})
print(user_prompt)

messages=[HumanMessage(content="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: ALL-in 코딩 공모전 수상작들을 요약해줘 \nContext: 스파르타 소식'AII-in 코딩 공모전’ 수상작을 소개합니다조회수  819·6분 분량2024. 9. 3.코딩은 더 이상 개발자만의 영역이 아닙니다. 누구나 아이디어만 있다면 창의적인 서비스를 만들어 세상을 바꿀 수 있습니다. 스파르타코딩클럽에서는 이러한 가능성을 믿고, 누구나 코딩을 통해 자신의 아이디어를 실현하고 실제 문제를 해결하는 경험을 쌓을 수 있도록 다양한 프로그램을 마련하고 있습니다.<All-in> 코딩 공모전은 대학생들이 캠퍼스에서 겪은 불편함과 문제를 자신만의 아이디어로 해결해보는 대회였는데요. 이번 공모전에서 다양한 혁신적인 아이디어와 열정으로 가득한 수많은 프로젝트가 탄생했습니다. 그중 뛰어난 성과를 낸 수상작 6개를 소개합니다.🏆\xa0대상[Lexi Note] 언어공부 필기 웹 서비스서비스 제작자: 다나와(김다애, 박나경)💡W는 어문학을 전공하는 대학생입니다. 매일 새로운 단어와 문장 구조를 공부하고 있지만, 효율적으로 학습하는 것이 쉽지 않았습니다. 단어의 의미를 찾기 위해 사전을 뒤적이고, 긴 문장을 이해하려고 번역기를 사용하다 보면, 필기 노트는 어느새 뒷전으로 밀려났거든요. 사전, 번역기, 원서, 필기노트를 왔다 갔다 하다 보면 시간이 다 지나가 버리곤 했죠.W와 같이 어문 전공생은 문법, 어휘, 문장 구조 등 다양한 자료를 학습해야 합니다. 여러 자료를 번갈아 학습하다보니 ‘

# 모델별 비교해 보기 (Pricing 기준)

모델명 (input 비용/ output 비용) Price per 1M tokens

## gpt-4o-mini ($0.15 / $0.60)

In [14]:
llm = ChatOpenAI(model="gpt-4o-mini")
response = llm.invoke(user_prompt)

# 깔끔하게 출력
formatted_response = response.content.replace(". ", ".\n\n")
print(formatted_response)

ALL-in 코딩 공모전에서 수상한 프로젝트들은 대학생들의 캠퍼스 문제를 해결하기 위한 혁신적인 아이디어로 구성되어 있습니다.

수상작 중 'Lexi Note'는 효율적인 언어 학습을 위한 웹 서비스이며, 'Crewing'은 동아리 정보 플랫폼으로 신입생들이 적합한 동아리를 찾도록 돕습니다.

'에코 클래스룸'은 학생과 교수 간의 실시간 소통을 개선하여 이해도를 파악할 수 있게 해주는 서비스입니다.


## gpt-4.1-mini ($0.40 / $1.60)

In [15]:
llm = ChatOpenAI(model="gpt-4.1-mini")
response = llm.invoke(user_prompt)

# 깔끔하게 출력
formatted_response = response.content.replace(". ", ".\n\n")
print(formatted_response)

ALL-in 코딩 공모전 수상작들은 대학생들이 캠퍼스 내 문제를 창의적으로 해결한 웹/앱 서비스들입니다.

대상작인 <Lexi Note>는 어문학 전공자의 언어 학습을 돕는 필기 통합 웹 서비스이고, 입선작 <Crewing>은 연합 동아리 정보 플랫폼, <학교생활 매니저>는 학교생활을 효율적으로 관리하는 서비스입니다.

우수상 <에코 클래스룸>은 익명 소통 기능을 통해 교수와 학생 간 실시간 피드백을 지원하는 수업 소통 서비스입니다.


## gpt-4.1-nano ($0.10 / $0.40)

In [16]:
llm = ChatOpenAI(model="gpt-4.1-nano")
response = llm.invoke(user_prompt)

# 깔끔하게 출력
formatted_response = response.content.replace(". ", ".\n\n")
print(formatted_response)

ALL-in 코딩 공모전 수상작은 대학생들이 캠퍼스 문제 해결을 위해 만든 웹/앱 서비스입니다.

대상작은 언어 학습을 돕는 'Lexi Note'로, 사전, 번역기, 필기 기능을 통합한 언어 학습 지원 서비스입니다.

입상작들은 동아리 정보 플랫폼, 학교생활 매니저, 실시간 수업 소통 서비스 등 캠퍼스 생활과 학습 환경을 개선하는 다양한 아이디어를 담고 있습니다.


# 결론

비용 측면에서 어떤것을 써야 할지 생각해 보았다.
nano는 너무 요약한거 같다.

4.1-mini와 4o-mini 2개가 성능이 비슷한데. 비용이 4o-mini가 더 싸니까 4o-mini 로 결정!